In [1]:
#%pip install ipyleaflet requests transformers torch
#%pip install 'accelerate>=0.26.0'
#%pip install -U bitsandbytes
# get access token from huggingface??

In [2]:
import requests
import torch
from IPython.display import display
from ipyleaflet import Map as LeafletMap, DrawControl
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer, GPTJForCausalLM

/Users/sophiakim/miniconda3/envs/virtual_env/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/Users/sophiakim/miniconda3/envs/virtual_env/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feed

In [3]:
# Once data has been processed, we check if there are any invalid data (i.e. marker is on land)
def is_valid_data(data):
    hourly_data = data['hourly']

    for hourly_entry in hourly_data.items():
        values = hourly_entry[1]
        if all(value is None for value in values):
            return False

    return True

In [4]:
# Initialize gpu for model to run on
device = torch.device("mps")

# Initialize llm
model_name = "EleutherAI/gpt-j-6B"
# pipe = pipeline(model=model_name, torch_dtype=torch.float16, tokenizer=model_name, batch_size=1)

# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, low_cpu_mem_usage=True)
model = GPTJForCausalLM.from_pretrained(
    "EleutherAI/gpt-j-6B",
    revision="float16",
    torch_dtype=torch.float16,
).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# pipe = pipeline(task="text-generation", model=model_name, tokenizer=tokenizer)

# model = GPT2LMHeadModel.from_pretrained(model_name)
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)



/Users/sophiakim/miniconda3/envs/virtual_env/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at EleutherAI/gpt-j-6B were not used when initializing GPTJForCausalLM: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.13.attn.bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.14.attn.bias', 'transformer.h.14.attn.masked_bias',

In [5]:
# Clean the data before passing to llm
def create_prompt(data):
    prompt = "Give me a time when it is safe to dock my ship based on the following data: \n"
    prompt += str(data)
    # clean_data += "Hours: " + hourly_data['time'][0] + " to " + hourly_data['time'][23] + "\n"
    # clean_data += "Latitude: " + str(data['latitude']) + ", Longitude " + str(data['longitude']) + "\n"
    # clean_data += "Hourly Wind Wave Heights: " + ", ".join([str(height) for height in data['hourly']['wind_wave_height']]) + " meters.\n"
    # clean_data += "Hourly Wind Wave Directions: " + ", ".join([str(direction) for direction in data['hourly']['wind_wave_direction']]) + " kts.\n"
    # clean_data += "Hourly Wind Wave Periods: " + ", ".join([str(period) for period in data['hourly']['wind_wave_period']]) + " s.\n"

    return prompt

In [6]:
def retrieve_summary(prompt):
    tokenizer.pad_token = tokenizer.eos_token
    input = tokenizer(prompt, return_tensors="pt", padding='longest')
    # adding pad token ID
    model.config.pad_token_id = model.config.eos_token_id
    # move input to gpu
    input = {key: value.to(device) for key, value in input.items()}
    # print("Inputs Shape: " + str(input['input_ids'].shape) + ", Device: " + str(input['input_ids'].device))
    # print(input)
    output = model.generate(
        input['input_ids'],
        attention_mask=input['attention_mask'],
        max_length=1000,
        num_return_sequences=1
    )
    print(output)
    result = tokenizer.decode(output[0], skip_special_tokens=True)
    # result = pipe(prompt, max_new_tokens=200)
    return result

In [11]:
# Event handler when user pins marker on map
def handle_marker(event, action, geo_json):
    coordinates = geo_json['geometry']['coordinates']
    longitude, latitude = str(coordinates[0]), str(coordinates[1])

    # Fetch sea conditions from Open-Meteo API
    url = "https://marine-api.open-meteo.com/v1/marine?latitude=" + latitude + "&longitude=" + longitude + "&hourly=wind_wave_height,wind_wave_direction,wind_wave_period&timezone=auto&forecast_days=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if is_valid_data(data):
            prompt = create_prompt(data)
            summary = retrieve_summary(prompt)
            print(summary[0]['generated_text'])
        else:
            print("Please try again")
    else:
        print("Error fetching data from Open-Meteo API: " + str(response.status_code))

In [10]:
map_center = [20.0, 0.0]
m = LeafletMap(center=map_center, zoom=2, scroll_wheel_zoom=True)

draw_control = DrawControl(
    marker={'shapeOptions': {}},
    polyline={},
    polygon={},
    circle={},
    circlemarker={},
)
m.add_control(draw_control)
draw_control.on_draw(handle_marker)

# Display map
display(m)

Map(center=[20.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


IndexError: tuple index out of range